# Software Vulnerability Detection using Deep Learning (Experiment Replication)

In [1]:
!install unrar

'install' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# !pip install transformers

In [5]:
!git clone https://github.com/danzz006/AVD.git

Cloning into 'AVD'...


In [1]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
# from torch.optim import AdamW as AdamW
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
import torch.nn.functional as F

import tensorflow as tf
# import mlflow


from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Available device: ", device)

c:\Users\Administrator\anaconda3\envs\avdbert\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Available device:  cuda:0


In [7]:
torch.cuda.is_available()

True

In [4]:
%cd AVD

c:\Users\Administrator\Downloads\AVD\AVD


In [5]:
!unrar e DataSet1000.rar

'unrar' is not recognized as an internal or external command,
operable program or batch file.


In [98]:
data=pd.read_csv("DataSet1000.csv")

In [99]:
data.tail()

,Name,Category,Impact,CWE-749,CWE-312,CWE-295,CWE-276,CWE-327,CWE-330,CWE-200,CWE-89,CWE-919,CWE-649,CWE-250
185268,private static String a(byte[] bArr) \n ...,1.0,NV,0,0,0,0,0,0,0,0,0,0,0
185269,"static void a(a.az r9, java.io.File r10) throw...",1.0,NV,0,0,0,0,0,0,0,0,0,0,0
185270,public static void a(Runnable runnable) \n ...,1.0,NV,0,0,0,0,0,0,0,0,0,0,0
185271,public static String c(Context context) \n ...,1.0,NV,0,0,0,0,0,0,0,0,0,0,0
185272,private static String d(Context context) \n ...,1.0,NV,0,0,0,0,0,0,0,0,0,0,0


In [12]:
os.getcwd()

'c:\\Users\\Administrator\\Downloads\\AVD'

In [2]:
data=pd.read_csv("DataSet_whnv.csv")

In [3]:
data.head()

,Name,Category,Impact,CWE-749,CWE-312,CWE-295,CWE-276,CWE-327,CWE-330,CWE-200,CWE-89,CWE-919,CWE-649,CWE-250
0,"public void addJavascriptObject(Object obj, St...",0,warning,1,0,0,0,0,0,0,0,0,0,0
1,public void initialize() \n setPlacemen...,0,warning,1,0,0,0,0,0,0,0,0,0,0
2,public void initialize() \n setPlacemen...,0,warning,1,0,0,0,0,0,0,0,0,0,0
3,public void displayRichMedia(final JSONObject ...,0,warning,1,0,0,0,0,0,0,0,0,0,0
4,this.webView.addJavascriptInterface(this.jsBri...,0,warning,1,0,0,0,0,0,0,0,0,0,0


In [4]:
for dataset in [data]:
    for col in range(2,14):
            dataset.iloc[:,col] = dataset.iloc[:,col].map({'False':0, 'True':1,False:0, True:1,'1':1,'0':0,'1.0':1,'0.0':0, 'NV':1, 'warning':0, 'high': 0})


In [5]:
data.head()

,Name,Category,Impact,CWE-749,CWE-312,CWE-295,CWE-276,CWE-327,CWE-330,CWE-200,CWE-89,CWE-919,CWE-649,CWE-250
0,"public void addJavascriptObject(Object obj, St...",0,0,1,0,0,0,0,0,0,0,0,0,0
1,public void initialize() \n setPlacemen...,0,0,1,0,0,0,0,0,0,0,0,0,0
2,public void initialize() \n setPlacemen...,0,0,1,0,0,0,0,0,0,0,0,0,0
3,public void displayRichMedia(final JSONObject ...,0,0,1,0,0,0,0,0,0,0,0,0,0
4,this.webView.addJavascriptInterface(this.jsBri...,0,0,1,0,0,0,0,0,0,0,0,0,0


In [6]:
del data['Category']


for col in data:
    if col != "Name":
        if data[col].value_counts()[1] < 1000:
            del data[col]
     

In [7]:
data.head()

,Name,Impact,CWE-312,CWE-276,CWE-327,CWE-200,CWE-89
0,"public void addJavascriptObject(Object obj, St...",0,0,0,0,0,0
1,public void initialize() \n setPlacemen...,0,0,0,0,0,0
2,public void initialize() \n setPlacemen...,0,0,0,0,0,0
3,public void displayRichMedia(final JSONObject ...,0,0,0,0,0,0
4,this.webView.addJavascriptInterface(this.jsBri...,0,0,0,0,0,0


In [8]:
del data['CWE-276']
del data['CWE-312']
del data['CWE-327']
del data['CWE-200']
del data['CWE-89']
data.to_csv("DataSet_bbnew.csv",index=False)

In [9]:
data.head()

,Name,Impact
0,"public void addJavascriptObject(Object obj, St...",0
1,public void initialize() \n setPlacemen...,0
2,public void initialize() \n setPlacemen...,0
3,public void displayRichMedia(final JSONObject ...,0
4,this.webView.addJavascriptInterface(this.jsBri...,0


In [10]:
print(data['Impact'].value_counts())

1    15999
0    15973
Name: Impact, dtype: int64


In [11]:
shuffled = data.sample(frac=1).reset_index(drop=True) # for randomizing data

In [12]:
shuffled.head()

,Name,Impact
0,"public long update(String userName, String pas...",0
1,public synchronized List<Identity> getIdentiti...,1
2,"private static final String OPT_OUT_KEY = ""opt...",0
3,public final WebResourceResponse shouldInterce...,0
4,public void reload() \n if (!(this.mSp ...,0


In [13]:
shuffled[shuffled.columns[0:5]]

,Name,Impact
0,"public long update(String userName, String pas...",0
1,public synchronized List<Identity> getIdentiti...,1
2,"private static final String OPT_OUT_KEY = ""opt...",0
3,public final WebResourceResponse shouldInterce...,0
4,public void reload() \n if (!(this.mSp ...,0
...,...,...
31967,"public static void a(boolean z, String str, Ob...",1
31968,public void onCreate(Bundle savedInstanceState...,0
31969,public final String toString() \n if (t...,0
31970,private final WebViewClient U = new WebViewCli...,0


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_tmp, x_test, y_tmp, y_test = train_test_split(shuffled["Name"], shuffled[shuffled.columns[1:5]], test_size=0.2, random_state=42)

In [16]:
x_train, x_val, y_train, y_val = train_test_split(x_tmp, y_tmp, test_size=0.1, random_state=42)

In [17]:
y_train.value_counts()

Impact
0         11521
1         11498
dtype: int64

In [18]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

In [19]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [20]:
# sample_txt = x_train[164212]

In [21]:
# sample_txt

In [23]:
# sample_txt = sample_txt.replace("\n", " ")
# sample_txt = sample_txt.replace(";", "")

In [24]:
# tokens = tokenizer.tokenize(sample_txt)
# token_ids = tokenizer.convert_tokens_to_ids(tokens)
# print(f' Sentence: {sample_txt}')
# print(f'   Tokens: {tokens}')
# print(f'Token IDs: {token_ids}')

In [25]:
# encoding = tokenizer.encode_plus(
#   sample_txt,
#   truncation=True,
#   max_length=32,
#   add_special_tokens=True, # Add '[CLS]' and '[SEP]'
#   return_token_type_ids=False,
#   padding='max_length',
#   return_attention_mask=True,
#   return_tensors='pt',  # Return PyTorch tensors
# )
# encoding.keys()

In [26]:
# print(len(encoding['input_ids'][0]))
# encoding['input_ids'][0]

In [27]:
# print(len(encoding['attention_mask'][0]))
# encoding['attention_mask']

In [28]:
# tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

In [29]:
# token_lens = []
# for txt in data.Name:
#   tokens = tokenizer.encode(txt, truncation=True, max_length=512)
#   token_lens.append(len(tokens))

In [30]:
# sns.histplot(token_lens)
# plt.xlim([0, 256]);
# plt.xlabel('Token count');

In [21]:
MAX_LEN = 500

In [22]:
class SWVulnerabilityDataset(Dataset):
    
  def __init__(self, code, targets, tokenizer, max_len):
    self.code = code
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
    
  def __len__(self):
    return len(self.code)

  def __getitem__(self, item):
    code = str(self.code[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      code,
      truncation=True,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'code_text': code,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [23]:
def create_data_loader(code, targets, tokenizer, max_len, batch_size):
  ds = SWVulnerabilityDataset(
    code=code,
    targets=targets,
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=8
  )

In [24]:
BATCH_SIZE = 4

train_data_loader = create_data_loader(x_train.to_numpy(), y_train.to_numpy(), tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(x_val.to_numpy(), y_val.to_numpy(), tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(x_test.to_numpy(), y_test.to_numpy(), tokenizer, MAX_LEN, BATCH_SIZE)

In [26]:
a = next(iter(train_data_loader))

In [24]:
# data = next(iter(train_data_loader))
# data.keys()

In [25]:
# print(data['input_ids'].shape)
# print(data['attention_mask'].shape)
# print(data['targets'].shape)

In [26]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
# last_hidden_state, pooled_output = bert_model(
#   input_ids=encoding['input_ids'],
#   attention_mask=encoding['attention_mask']
# )[0:]

In [28]:
# last_hidden_state.shape, pooled_output.shape

In [29]:
class SWVulnerabilityClassifier(nn.Module):
  def __init__(self, n_classes):
    super(SWVulnerabilityClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
#     self.softmax = F.softmax()
    
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )[0:]
    output = self.drop(pooled_output)
    return F.softmax(self.out(output))

In [30]:
model = SWVulnerabilityClassifier(len(y_train.columns))
model = model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
# input_ids = data['input_ids'].to(device)
# attention_mask = data['attention_mask'].to(device)
# print(input_ids.shape) # batch size x seq length
# print(attention_mask.shape) # batch size x seq length

In [32]:
EPOCHS = 50
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

c:\Users\Administrator\anaconda3\envs\avdbert\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [40]:
device

device(type='cuda', index=0)

In [37]:
model = model.train()

In [33]:
batch_data = next(iter(train_data_loader))

In [41]:
losses = []
correct_predictions = 0

history = defaultdict(list)

for epoch in range(EPOCHS):
    model = model.train()
    batch_data = next(iter(train_data_loader))
    input_ids = batch_data["input_ids"].to(device)
    attention_mask = batch_data["attention_mask"].to(device)
    targets = batch_data["targets"].float().to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    
    targets_arr = []
    for i in targets:
        targets_arr.append(np.argmax(i.cpu()).numpy().item())
    targets = torch.Tensor(targets_arr).to(device)
    
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    
    train_acc = correct_predictions.double() / len(train_data_loader)
    train_loss = np.mean(losses)
    print("*"*40)
    print(f"Epoch: {epoch}")
    print("Training accuracy: ", train_acc.item())
    print("Training loss: ", train_loss)

    
    history['train_acc'].append(train_acc.item())
    history['train_loss'].append(train_loss)
    
    print("Evaluating model..")
    model = model.eval()
    val_losses = []
    val_correct_predictions = 0
    with torch.no_grad():
        val_data = next(iter(val_data_loader))
        input_ids = val_data["input_ids"].to(device)
        attention_mask = val_data["attention_mask"].to(device)
        targets = val_data["targets"].float().to(device)
        outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
        )
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        
        targets_arr = []
        for i in targets:
            targets_arr.append(np.argmax(i.cpu()).numpy().item())
        targets = torch.Tensor(targets_arr).to(device)
        
        val_correct_predictions += torch.sum(preds == targets)
        val_losses.append(loss.item())
        
        val_acc = val_correct_predictions.double() / len(val_data_loader)
        val_loss = np.mean(val_losses)
        
    print("Eval accuracy: ", val_acc.item())
    print("Eval loss: ", val_loss)
    print("*"*40)

    history['val_acc'].append(val_acc.item())
    history['val_loss'].append(val_loss)
    
    

In [ ]:
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')
plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1])